In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [2]:
def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model")

# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_model")

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 1s 2ms/step - loss: 0.2814
INFO:tensorflow:Assets written to: my_model\assets
4/4 [==============================] - 0s 2ms/step - loss: 0.2716


In [4]:
!dir my_model

 Volume in drive D is Data
 Volume Serial Number is 049D-8712

 Directory of D:\repositories\mMachine-Learning\src\my_model

2022/09/20  17:23    <DIR>          .
2022/09/20  17:23    <DIR>          ..
2022/09/20  17:23    <DIR>          assets
2022/09/20  17:23             4,775 keras_metadata.pb
2022/09/20  17:23            46,340 saved_model.pb
2022/09/20  17:23    <DIR>          variables
               2 File(s)         51,115 bytes
               4 Dir(s)  137,763,495,936 bytes free


In [5]:
class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        self.hidden_units = hidden_units
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x

    def get_config(self):
        return {"hidden_units": self.hidden_units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)


model = CustomModel([16, 16, 10])
# Build the model by calling it
input_arr = tf.random.uniform((1, 5))
outputs = model(input_arr)
model.save("my_model")

# Option 1: Load with the custom_object argument.
loaded_1 = keras.models.load_model(
    "my_model", custom_objects={"CustomModel": CustomModel}
)

# Option 2: Load without the CustomModel class.

# Delete the custom-defined model class to ensure that the loader does not have
# access to it.
del CustomModel

loaded_2 = keras.models.load_model("my_model")
np.testing.assert_allclose(loaded_1(input_arr), outputs)
np.testing.assert_allclose(loaded_2(input_arr), outputs)

print("Original model:", model)
print("Model Loaded with custom objects:", loaded_1)
print("Model loaded without the custom object class:", loaded_2)

INFO:tensorflow:Assets written to: my_model\assets
Original model: <__main__.CustomModel object at 0x0000019ED44D9F70>
Model Loaded with custom objects: <__main__.CustomModel object at 0x0000019EE31BA370>
Model loaded without the custom object class: <keras.saving.saved_model.load.CustomModel object at 0x0000019EE4DE9B80>
